## This notebook runs the Ring All Reduce (RAR) architecture _with_ validation partition of the data.

In [ ]:
import os  # Operating system functionalities
import numpy as np  # Numerical operations and array handling
import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt  # Data visualization
import pickle  # Serialization and deserialization of Python objects
import random as rn  # Random number generator
import itertools  # Functions for creating iterators for efficient looping
from IPython.display import display  # Function for rendering in Jupyter Notebook

from sklearn.model_selection import train_test_split  # Splitting data into training and testing sets
from sklearn.preprocessing import MinMaxScaler  # Scaling features to a range

import tensorflow as tf  # Main machine learning framework
from tensorflow.keras.layers import BatchNormalization  # Normalizing layer in neural networks
import tensorflow.keras as keras  # High-level neural networks API
from keras import regularizers  # Regularization techniques in Keras
from keras.layers import Dropout  # Dropout regularization

Seeds to control randomness in the process.

In [ ]:
np.random.seed(0)  # Setting the random seed for reproducibility
rn.seed(0)  # Setting the random seed for reproducibility
tf.random.set_seed(0) # Setting the TensorFlow random seed for reproducibility

Loading processed and curated dataset:

In [ ]:
dfHeart = pd.read_csv("/srv/heart_ConditionalMeanImputation.csv")
dfHeart

Assigning the name of the originating hospital to each point in the dataset.

In [ ]:
dfHeart["Name"]= "name"
# Sliced from original raw data
dfHeart.loc[0:293,"Name"] = "hung" # Hungarian Institute of Cardiology, Budapest
# Author: Andras Janosi, M.D.
dfHeart.loc[293:477,"Name"] = "swit" # University Hospital (Zurich and Basel) Switzerland
# Author: William Steinbrunn, M.D. and Matthias Pfisterer, M.D respectively.
dfHeart.loc[477:615,"Name"] = "long" # V.A. Medical Center, Long Beach
# Author: Robert Detrano, M.D., Ph.D.
dfHeart.loc[615:766,"Name"] = "stat" # Statlog dataset (confidential source)
# Author: "King RD. Department of Statistics and Modelling Science, University of Strathclyde, Glasgow"
dfHeart.loc[766:918,"Name"] = "clev" # Cleveland Clinic Foundation
# Author: Robert Detrano, M.D., Ph.D.

Creating different clients with the sliced data.

In [ ]:
data_dict = {}

groups = dfHeart.groupby('Name')

for idx, group in enumerate(groups):
    X = group[1].drop(['HeartDisease', 'Name'], axis=1)
    y = group[1]['HeartDisease']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    
    # Scaling the data
    scaler = MinMaxScaler()
    scaler = scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Creating a dictionary for each client
    data_dict[idx] = {
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train,
        'y_test': y_test
    }

Building the optimal ANN architecture, previously estimated.

In [ ]:
# Create the model
model = keras.models.Sequential()

shape = data_dict[list(data_dict.keys())[0]]['X_train'].shape[1]

# Input layer
model.add(keras.layers.Dense(units=256, input_shape=(shape,), activation="relu", kernel_regularizer=regularizers.l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Add a Dropout layer with a dropout rate of 50%

# Hidden layers
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Add a Dropout layer with a dropout rate of 50%

model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Add a Dropout layer with a dropout rate of 50%

model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Add a Dropout layer with a dropout rate of 50%

model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Add a Dropout layer with a dropout rate of 50%

# Output layer
model.add(keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss="binary_crossentropy", 
              metrics=["accuracy", "AUC"])

In [ ]:
def ave_weights(n_i,listOfWeights):
    """
    Aggregation function

   :param list n_i: Number of samples for each client
   :param list listOfWeights: Weights for each client
   :return: Final weighted average for global model
    """
    N = sum(n_i) # total number of samples of all clients
    # initial weights of global model, set to zero
    ave_weights = listOfWeights[0]
    ave_weights = [i * 0 for i in ave_weights]
    # loop whose range is number of clients
    for j in range(len(n_i)):
        # receive weights from clients
        rec_weight = listOfWeights[j]
        # multiply the client weights by number of local data samples in client local data
        rec_weight =  [i * n_i[j] for i in rec_weight]
        # divide the weights by total number of samples of all clients
        rec_weight =  [i / N for i in rec_weight]
        # sum the weights of new client with the prior
        ave_weights = [x + y for x, y in zip(ave_weights,rec_weight)]
    return ave_weights

Ring all reduce architecture with 5 clients: [5,1,2,3,4] -> [1,2,3,4,5]

In [ ]:
clients_send = list(range(1,len(data_dict)+1))
clients_send

In [ ]:
clients = clients_send.copy()

In [ ]:
clients.insert(0, clients.pop())
print(clients) 

The weights are sent from clients to clients_send: [5,1,2,3,4] -> [1,2,3,4,5] <br>
Client 1 will possess a model (formed by aggregating the weights of clients 5 and 1), <br>
which will differ from client 2 (formed by the weights of clients 1 and 2) <br>
client_send[i] could even be a list of clients <br>

Initialization of empty lists. 

In [ ]:
# Metrics for validation
loss_val_round = [[] for _ in range(len(clients))] 
acc_val_round = [[] for _ in range(len(clients))]  
AUC_val_round = [[] for _ in range(len(clients))] 
# Metrics for test
acc_test_round = [[] for _ in range(len(clients))] 
loss_test_round = [[] for _ in range(len(clients))]
AUC_test_round = [[] for _ in range(len(clients))]
# To save client weights of certain rounds
save_weights = []

Running the rounds.

In [ ]:
# Maximum number of rounds
n_times = 100

for i in range(1,n_times+1):
    # Each round begins with one weight per client.
    # weights_clients will save the weights for each client INSIDE a round
    initial_weights = model.get_weights()
    weights_clients = [[initial_weights] for _ in range(len(clients))]
    # To save validation metrics INSIDE a round
    loss_val_clients = []
    acc_val_clients = []
    AUC_val_clients = []
    
    for idx in range(len(clients)):
        # The model is trained with the weights (initial in the first round or averaged later)
        model.set_weights(weights_clients[idx][-1])
        X_train_np = np.array(data_dict[idx]['X_train'])
        y_train_np = np.array(data_dict[idx]['y_train'])
        history = model.fit(X_train_np,y_train_np, epochs=38, batch_size=14, validation_split=0.15, verbose=0)
        
        # The new weights are added to the list stored at position i of weights_clients
        weights_clients[clients[idx]-1].append(model.get_weights())
    
        # Add those weights to a list stored in position 'j', 
        # where 'j' represents all the clients with whom client 'i' communicates
        weights_clients[clients_send[idx]-1].append(model.get_weights())
        loss_val_clients.append(history.history['val_loss'][-1])
        acc_val_clients.append(history.history['val_accuracy'][-1])
        AUC_val_clients.append(history.history['val_auc'][-1])
        
    # Now there are two sets of weights per position in weights_clients. 
    # weights_clients = [[weights_clients1, weights_clients5], [weights_clients2, weights_clients1], 
    # [weights_clients3, weights_clients2], [weights_clients4, weights_clients3],
    # [weights_clients5, weights_clients5]]
    
    # Averaging them will provide one set of weights again for each position in weights_clients.
    for idx in range(len(clients)):
        # Calculating the lengths
        samples_len = [len(data_dict[clients[idx]-1]['X_train']),len(data_dict[clients_send[idx]-1]['X_train'])]
        # Aggregate the weights obtained with each client using an aggregation function that performs a weighted average:
        avg_weights = ave_weights(samples_len,weights_clients[idx]) # use of aggregation function
        weights_clients[idx] = [avg_weights]
        model.set_weights(avg_weights)
        X_test_np = np.array(data_dict[idx]['X_test'])
        y_test_np = np.array(data_dict[idx]['y_test'])
        score_test = model.evaluate(X_test_np, y_test_np, verbose = 0)
        # Saving metrics for validation:
        loss_val_round[idx].append(loss_val_clients[idx])
        acc_val_round[idx].append(acc_val_clients[idx])
        AUC_val_round[idx].append(AUC_val_clients[idx])
        # Saving metrics for test:
        loss_test_round[idx].append(score_test[0])
        acc_test_round[idx].append(score_test[1])
        AUC_test_round[idx].append(score_test[2])
    if i % 5 == 0: # Save weights_clients each 5 rounds (less storage space)
        save_weights.append(weights_clients)

Now, let's plot the evolution (in rounds) of validation metrics per client for the RAR architecture.

In [ ]:
loss_val_round_np = np.array(loss_val_round)
# Create a figure with three subplots (one row and three columns)
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8, 12))

# Iterate over the lists of loss, accuracy, and AUC of each client
for i, group in enumerate(groups):
    name = group[0]
    client_loss = loss_val_round_np[i]
    client_accuracy = acc_val_round[i]
    client_auc = AUC_val_round[i]
    
    # Subplot 1: Loss Curve
    ax1.plot(range(1, n_times + 1), client_loss / max(client_loss), label= f'Client {i+1}: {name}', alpha=0.7)

    # Subplot 2: Accuracy Curve
    ax2.plot(range(1, n_times + 1), client_accuracy, label= f'Client {i+1}: {name}', alpha=0.7)

    # Subplot 3: AUC Curve
    ax3.plot(range(1, n_times + 1), client_auc, label= f'Client {i+1}: {name}', alpha=0.7)

# Add labels and titles to subplots
ax1.set_xlabel('Round')
ax1.set_ylabel('Normalized Average Loss')
ax1.set_title('Normalized Average Loss (validation)')
ax1.grid()
ax1.legend()

ax2.set_xlabel('Round')
ax2.set_ylabel('Average Accuracy')
ax2.set_title('Normalized Average Accuracy (validation)')
ax2.grid()
ax2.legend()

ax3.set_xlabel('Round')
ax3.set_ylabel('AUC')
ax3.set_title('Area Under Curve (AUC - validation)')
ax3.grid()
ax3.legend()

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

Plotting the evolution of test metrics per client.

In [ ]:
loss_test_round_np = np.array(loss_test_round)
# Create a figure with three subplots (one row and three columns)
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8, 12))

# Iterate over the lists of loss, accuracy, and AUC of each client
for i, group in enumerate(groups):
    client_loss = loss_test_round_np[i]
    client_accuracy = acc_test_round[i]
    client_auc = AUC_test_round[i]
    
    # Subplot 1: Loss Curve
    ax1.plot(range(1, n_times + 1), client_loss / max(client_loss), label= f'Client {i+1}: {name}', alpha=0.7)

    # Subplot 2: Accuracy Curve
    ax2.plot(range(1, n_times + 1), client_accuracy, label= f'Client {i+1}: {name}', alpha=0.7)

    # Subplot 3: AUC Curve
    ax3.plot(range(1, n_times + 1), client_auc, label= f'Client {i+1}: {name}', alpha=0.7)

# Add labels and titles to subplots
ax1.set_xlabel('Round')
ax1.set_ylabel('Normalized Average Loss')
ax1.set_title('Normalized Average Loss (test)')
#ax1.set(xlim=(0, 25))
ax1.grid()
ax1.legend()

ax2.set_xlabel('Round')
ax2.set_ylabel('Average Accuracy')
ax2.set_title('Normalized Average Accuracy (test)')
# ax2.set(xlim=(0, 25))
ax2.grid()
ax2.legend()

ax3.set_xlabel('Round')
ax3.set_ylabel('AUC')
ax3.set_title('Area Under Curve (AUC)')
# ax3.set(xlim=(0, 25))
ax3.grid()
ax3.legend()

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

Saving important lists for later analysis.

In [ ]:
# Combine the lists of lists into a dictionary
data_to_save = {
    'acc_val_round': acc_val_round,
    'loss_val_round': loss_val_round,
    'AUC_val_round': AUC_val_round,
    'acc_test_round': acc_test_round,
    'loss_test_round': loss_test_round,
    'AUC_test_round': AUC_test_round,
    'save_weights': save_weights
}

# Save the combined data using Pickle
with open('results_Test_RAR_v3_validation.pkl', 'wb') as file:
    pickle.dump(data_to_save, file)

Calculation of the weight divergence between pairs of clients for rounds 70 or 80, where loss metric converges.

In [ ]:
def weight_divergence(weights_m, weights_n):
    """
    Compute the weight divergence between two sets of weights.

    :param list weights_m: Set of weights for model m
    :param list weights_n: Set of weights for model n
    :return: Weight divergence between the two sets of weights
    """
    # Flatten the weights arrays to 1-dimensional arrays
    flat_weights_m = np.concatenate([w.ravel() for w in weights_m])
    flat_weights_n = np.concatenate([w.ravel() for w in weights_n])

    # Compute the Euclidean norm of the difference between weights_m and weights_n
    norm_diff = np.linalg.norm(flat_weights_m - flat_weights_n)

    # Compute the Euclidean norm of weights_n
    norm_n = np.linalg.norm(flat_weights_n)

    # Calculate the weight divergence
    divergence = norm_diff / norm_n

    return divergence

In [ ]:
# Calculate weight divergence for all combinations of pairs of clients
divergences = []
for pair in itertools.combinations(range(5), 2):
    client_a = pair[0]
    client_b = pair[1]
    divergence = weight_divergence(save_weights[14][client_a][0], save_weights[14][client_b][0])
    divergences.append({'Pair of Clients': f'{client_a + 1}-{client_b + 1}', 'Weight Divergence': divergence})

# Creating the table
divergences_df = pd.DataFrame(divergences)
divergences_df['Weight Divergence'] = divergences_df['Weight Divergence'].apply(lambda x: f'{x:.2f}')
divergences_df.index += 1
weight_divergence_table = divergences_df.style.set_caption('Weight Divergence in Round 70')

# Display the table
display(weight_divergence_table)

In [ ]:
# Calculate weight divergence for all combinations of pairs of clients
divergences = []
for pair in itertools.combinations(range(5), 2):
    client_a = pair[0]
    client_b = pair[1]
    divergence = weight_divergence(save_weights[16][client_a][0], save_weights[16][client_b][0])
    divergences.append({'Pair of Clients': f'{client_a + 1}-{client_b + 1}', 'Weight Divergence': divergence})

# Creating the table
divergences_df = pd.DataFrame(divergences)
divergences_df['Weight Divergence'] = divergences_df['Weight Divergence'].apply(lambda x: f'{x:.2f}')
divergences_df.index += 1
weight_divergence_table = divergences_df.style.set_caption('Weight Divergence in Round 80')

# Display the table
display(weight_divergence_table)